# Домашнее задание № 1
Используемые библиотеки

In [156]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing
import random

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [55]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
# Добавляем заголовок в запрос - иначе сайт выдает ошибку 403
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
s = requests.get(url, headers=headers).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
### Если не работает загрузка on-line
# df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


Вариант решения с простой интерпретацией со значениями: "Ниже нормы", "В пределах нормы", "Выше нормы" с использованием цикла for

In [101]:

# Создадим столбец с нижней границей 
df['Нижняя граница'] = np.NaN
# Создадим столбец с верхней границей 
df['Верхняя граница'] = np.NaN

# Для каждой строки в датафрейме
for i in range(len(df)):
    
    # Если в нормативе содержится диапазон (в пределах) - нужно взять его нижнюю границу
    if 'пределах' in df.at[i,'Норматив'].lower():
        # Разбиваем значение ячейки по пробелу, берем последний элемент (диапазон значений)
        val_cell = df.at[i,'Норматив'].split(' ')[-1]
        # В нормативе разделители запятые (в нужных строках значения целые, но могут измениться)
        try: val_cell = val_cell.replace(',','.')
        except SyntaxError: pass
        # Разбиваем значение по дефису, первое значение включаем в нижнюю границу, второе в верхнюю
        df.at[i,'Нижняя граница'] = val_cell.split('-')[0]
        df.at[i,'Верхняя граница'] = val_cell.split('-')[1]
        
    # Если в нормативе содержится верхняя граница (не более) - включаем его в верхнюю границу
    if 'не более' in df.at[i,'Норматив'].lower():
        # Разбиваем значение ячейки по пробелу, берем последний элемент (значение или диапазон значений)
        val_cell = df.at[i,'Норматив'].split(' ')[-1]
        # В нормативе разделители запятые (в нужных строках значения целые, но могут измениться)
        try: val_cell = val_cell.replace(',','.')
        except SyntaxError: pass
        # В части нормативов верхняя граница указана диапазоном - берем крайнюю верхнюю границу
        df.at[i,'Верхняя граница'] = val_cell.split('-')[-1]

# Так как граничные значения показателей не могут быть отрицательными - заполним пропуски нулями
df['Нижняя граница'].fillna(0, inplace=True)
df['Верхняя граница'].fillna(0, inplace=True)

# Приводим гриницы показателей к типу float
df['Нижняя граница'] = df['Нижняя граница'].astype(float)
df['Верхняя граница'] = df['Верхняя граница'].astype(float)

df

,Показатель,Единица измерений,Результат анализа,Норматив,Нижняя граница,Верхняя граница
0,pH,единицы pH,8.4,в пределах 6-9,6.0,9.0
1,Запах,баллы,1,не более 2-3,0.0,3.0
2,Цветность,градусы,б/цвета,не более 30,0.0,30.0
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,7.0,10.0
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",0.0,1.5
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0.0,3.3
6,Нитраты (по NO3),мг/дм3,24,не более 45,0.0,45.0
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5",0.0,3.5
8,Хлориды (Cl),мг/дм3,200,не более 350,0.0,350.0
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,0.0,500.0


Интерпретируем результаты

In [108]:
# Добавим столбец с результатом сравнения
df['Интерпретация результата'] = np.NaN

# Для каждой строки в датафрейме
for i in range(len(df)):
    # В результате анализа имеется значение б/цвета, которое соответствует нулю - заменим текст на 0
    if df.at[i,'Результат анализа'] == 'б/цвета':
        df.at[i,'Результат анализа'] = 0
    # Приведем значения столбца "Результат анализа" к типу float
    df.at[i,'Результат анализа'] = float(df.at[i,'Результат анализа'])

    # Проверяем на вхождение результата анализа в допустимые границы
    if (df.at[i,'Результат анализа'] >= df.at[i,'Нижняя граница']) and (df.at[i,'Результат анализа'] <= df.at[i,'Верхняя граница']):
        # Результат интерпретируем как "В пределах нормы"
        df.at[i,'Интерпретация результата'] = 'В пределах нормы'

    # Если результат анализа выше верхней границы
    elif df.at[i,'Результат анализа'] > df.at[i,'Верхняя граница']:
        # Результат интерпретируем как "Ниже нормы"
        df.at[i,'Интерпретация результата'] = 'Выше нормы'

    # Если результат анализа меньше нижней границы
    elif df.at[i,'Результат анализа'] < df.at[i,'Нижняя граница']:
        # Результат интерпретируем как "Ниже нормы"
        df.at[i,'Интерпретация результата'] = 'Ниже нормы'
df

,Показатель,Единица измерений,Результат анализа,Норматив,Нижняя граница,Верхняя граница,Интерпретация результата
0,pH,единицы pH,8.4,в пределах 6-9,6.0,9.0,В пределах нормы
1,Запах,баллы,1.0,не более 2-3,0.0,3.0,В пределах нормы
2,Цветность,градусы,0.0,не более 30,0.0,30.0,В пределах нормы
3,Жёсткость,мг-эквл/дм3,9.2,в пределах 7-10,7.0,10.0,В пределах нормы
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",0.0,1.5,В пределах нормы
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0.0,3.3,В пределах нормы
6,Нитраты (по NO3),мг/дм3,24.0,не более 45,0.0,45.0,В пределах нормы
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5",0.0,3.5,В пределах нормы
8,Хлориды (Cl),мг/дм3,200.0,не более 350,0.0,350.0,В пределах нормы
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,0.0,500.0,В пределах нормы


In [110]:
# Устанавливаем показатель индексом
df.set_index('Показатель',inplace=True)

# Столбцы с границами нормы можно удалить
del df['Нижняя граница']
del df['Верхняя граница']
df


,Единица измерений,Результат анализа,Норматив,Интерпретация результата
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,В пределах нормы
Запах,баллы,1.0,не более 2-3,В пределах нормы
Цветность,градусы,0.0,не более 30,В пределах нормы
Жёсткость,мг-эквл/дм3,9.2,в пределах 7-10,В пределах нормы
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",В пределах нормы
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",В пределах нормы
Нитраты (по NO3),мг/дм3,24.0,не более 45,В пределах нормы
Фосфаты (P),мг/дм3,0.36,"не более 3,5",В пределах нормы
Хлориды (Cl),мг/дм3,200.0,не более 350,В пределах нормы


Усложненный вариант интерпретации со значениями: "Ниже нормы", "В пределах нижней границы нормы", "В пределах нормы", "В пределах верхней границы нормы", "Выше нормы"

In [136]:
# Получаем датафрейм

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
# Добавляем заголовок в запрос - иначе сайт выдает ошибку 403
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
s = requests.get(url, headers=headers).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
### Если не работает загрузка on-line
# df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


Создадим столбцы с границами нормы

In [139]:
# Создадим столбец с крайней нижней границей 
df['Нижняя крайняя граница'] = np.NaN
# Создадим столбец с комфортной нижней границей 
df['Нижняя оптимальная граница'] = np.NaN
# Создадим столбец с комфортной верхней границей 
df['Верхняя оптимальная граница'] = np.NaN
# Создадим столбец с крайней верхней границей 
df['Верхняя крайняя граница'] = np.NaN

# Для каждой строки в датафрейме
for i in range(len(df)):
    
    # Если в нормативе содержится диапазон (в пределах) - нужно взять его нижнюю границу
    if 'пределах' in df.at[i,'Норматив'].lower():
        # Разбиваем значение ячейки по пробелу, берем последний элемент (диапазон значений)
        val_cell = df.at[i,'Норматив'].split(' ')[-1]
        # В нормативе разделители запятые (в нужных строках значения целые, но могут измениться)
        try: val_cell = val_cell.replace(',','.')
        except SyntaxError: pass
        # Разбиваем значение по дефису, первое значение включаем в нижние границы, второе в верхние
        df.at[i,'Нижняя крайняя граница'] = val_cell.split('-')[0]
        df.at[i,'Нижняя оптимальная граница'] = val_cell.split('-')[0]
        df.at[i,'Верхняя оптимальная граница'] = val_cell.split('-')[1]
        df.at[i,'Верхняя крайняя граница'] = val_cell.split('-')[1]
        
    # Если в нормативе содержится верхняя граница (не более) - включаем его в верхнюю границу
    if 'не более' in df.at[i,'Норматив'].lower():
        # Разбиваем значение ячейки по пробелу, берем последний элемент (значение или диапазон значений)
        val_cell = df.at[i,'Норматив'].split(' ')[-1]
        # В нормативе разделители запятые (в нужных строках значения целые, но могут измениться)
        try: val_cell = val_cell.replace(',','.')
        except SyntaxError: pass
        # В части нормативов верхняя граница указана диапазоном - нижняя границу берем в оптимальную верхнюю границу, верхнюю -в крайнюю верхнюю границу
        df.at[i,'Верхняя оптимальная граница'] = val_cell.split('-')[0]
        df.at[i,'Верхняя крайняя граница'] = val_cell.split('-')[-1]

# Так как граничные значения показателей не могут быть отрицательными - заполним пропуски нулями'
df['Нижняя крайняя граница'].fillna(0, inplace=True)
df['Нижняя оптимальная граница'].fillna(0, inplace=True)
df['Верхняя оптимальная граница'].fillna(0, inplace=True)
df['Верхняя крайняя граница'].fillna(0, inplace=True)

# Приводим гриницы показателей к типу float
df['Нижняя крайняя граница'] = df['Нижняя крайняя граница'].astype(float)
df['Нижняя оптимальная граница'] = df['Нижняя оптимальная граница'].astype(float)
df['Верхняя оптимальная граница'] = df['Верхняя оптимальная граница'].astype(float)
df['Верхняя крайняя граница'] = df['Верхняя крайняя граница'].astype(float)

df

,Показатель,Единица измерений,Результат анализа,Норматив,Нижняя крайняя граница,Нижняя оптимальная граница,Верхняя оптимальная граница,Верхняя крайняя граница,Интерпретация результата
0,pH,единицы pH,8.4,в пределах 6-9,6.0,6.0,9.0,9.0,NaN
1,Запах,баллы,1,не более 2-3,0.0,0.0,2.0,3.0,NaN
2,Цветность,градусы,б/цвета,не более 30,0.0,0.0,30.0,30.0,NaN
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,7.0,7.0,10.0,10.0,NaN
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",0.0,0.0,1.5,1.5,NaN
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0.0,0.0,3.3,3.3,NaN
6,Нитраты (по NO3),мг/дм3,24,не более 45,0.0,0.0,45.0,45.0,NaN
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5",0.0,0.0,3.5,3.5,NaN
8,Хлориды (Cl),мг/дм3,200,не более 350,0.0,0.0,350.0,350.0,NaN
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,0.0,0.0,500.0,500.0,NaN


Интерпретируем результаты

In [152]:
# Добавим столбец с результатом сравнения
df['Интерпретация результата'] = np.NaN

# Для каждой строки в датафрейме
for i in range(len(df)):
    # В результате анализа имеется значение б/цвета, которое соответствует нулю - заменим текст на 0
    if df.at[i,'Результат анализа'] == 'б/цвета':
        df.at[i,'Результат анализа'] = 0
    # Приведем значения столбца "Результат анализа" к типу float
    df.at[i,'Результат анализа'] = float(df.at[i,'Результат анализа'])

    # Проверяем на вхождение результата анализа в оптимальные границы
    if (df.at[i,'Результат анализа'] >= df.at[i,'Нижняя оптимальная граница']) and (df.at[i,'Результат анализа'] <= df.at[i,'Верхняя оптимальная граница']):
        # Результат интерпретируем как "В пределах нормы"
        df.at[i,'Интерпретация результата'] = 'В пределах нормы'

    # Проверяем на вхождение результата анализа в верхние допустимые границы
    elif (df.at[i,'Результат анализа'] > df.at[i,'Верхняя оптимальная граница']) and (df.at[i,'Результат анализа'] <= df.at[i,'Верхняя крайняя граница']):
        # Результат интерпретируем как "В пределах верхней нормы"
        df.at[i,'Интерпретация результата'] = 'В пределах верхней нормы'

    # Проверяем на вхождение результата анализа в нижние допустимые границы
    elif (df.at[i,'Результат анализа'] >= df.at[i,'Нижняя крайняя граница']) and (df.at[i,'Результат анализа'] < df.at[i,'Нижняя оптимальная граница']):
        # Результат интерпретируем как "В пределах нижней нормы"
        df.at[i,'Интерпретация результата'] = 'В пределах нижней нормы'

    # Если результат анализа выше крайней верхней границы
    elif df.at[i,'Результат анализа'] > df.at[i,'Верхняя крайняя граница']:
        # Результат интерпретируем как "Ниже нормы"
        df.at[i,'Интерпретация результата'] = 'Выше нормы'

    # Если результат анализа меньше крайней нижней границы
    elif df.at[i,'Результат анализа'] < df.at[i,'Нижняя крайняя граница']:
        # Результат интерпретируем как "Ниже нормы"
        df.at[i,'Интерпретация результата'] = 'Ниже нормы'
df

,Показатель,Единица измерений,Результат анализа,Норматив,Нижняя крайняя граница,Нижняя оптимальная граница,Верхняя оптимальная граница,Верхняя крайняя граница,Интерпретация результата
0,pH,единицы pH,8.4,в пределах 6-9,6.0,6.0,9.0,9.0,В пределах нормы
1,Запах,баллы,1.0,не более 2-3,0.0,0.0,2.0,3.0,В пределах нормы
2,Цветность,градусы,0.0,не более 30,0.0,0.0,30.0,30.0,В пределах нормы
3,Жёсткость,мг-эквл/дм3,9.2,в пределах 7-10,7.0,7.0,10.0,10.0,В пределах нормы
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",0.0,0.0,1.5,1.5,В пределах нормы
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0.0,0.0,3.3,3.3,В пределах нормы
6,Нитраты (по NO3),мг/дм3,24.0,не более 45,0.0,0.0,45.0,45.0,В пределах нормы
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5",0.0,0.0,3.5,3.5,В пределах нормы
8,Хлориды (Cl),мг/дм3,200.0,не более 350,0.0,0.0,350.0,350.0,В пределах нормы
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,0.0,0.0,500.0,500.0,В пределах нормы


Приводим датафрейм к читабельному виду

In [154]:
# Устанавливаем показатель индексом
df.set_index('Показатель',inplace=True)

# Столбцы с границами нормы можно удалить
del df['Нижняя крайняя граница']
del df['Нижняя оптимальная граница']
del df['Верхняя оптимальная граница']
del df['Верхняя крайняя граница']
df

,Единица измерений,Результат анализа,Норматив,Интерпретация результата
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,В пределах нормы
Запах,баллы,1.0,не более 2-3,В пределах нормы
Цветность,градусы,0.0,не более 30,В пределах нормы
Жёсткость,мг-эквл/дм3,9.2,в пределах 7-10,В пределах нормы
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",В пределах нормы
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",В пределах нормы
Нитраты (по NO3),мг/дм3,24.0,не более 45,В пределах нормы
Фосфаты (P),мг/дм3,0.36,"не более 3,5",В пределах нормы
Хлориды (Cl),мг/дм3,200.0,не более 350,В пределах нормы


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


### Аналитическое решени задачи

Вероятность вытащить первый раз апельсин (из 5 апельсинов и 4 яблок): $P(O_1)=\frac{5}{5+4}=\frac{5}{9}$

Вероятность вытащить второй раз подряд апельсин (из 4 апельсинов и 4 яблок): $P(O_2)=\frac{4}{4+4}=\frac{2}{8}$

Вероятность вытащить третий раз подряд апельсин (из 3 апельсинов и 4 яблок): $P(O_3)=\frac{3}{3+4}=\frac{3}{7}$

Вероятность вытащить 3 апельсина подряд: $P(O_r)=\frac{5}{9}*\frac{4}{8}*\frac{3}{7}=\frac{5*4*3}{9*8*7}=\frac{60}{504}=\frac{5}{42}=0.119$

### Экспериментальное подтверждение

In [192]:
# Задаем число экспериментов
n = 100000

# Список результатов экспериментов
result = []

# генерируем выборку
for i in range(n): 
    # На начало эксперимента в ящике 5 апельсинов и 4 яблока
    box = ['orange']*5 + ['apple']*4
    # Апельсинов пока не вытащили
    oranges = 0
    # Вытаскиваем 3 фрукта
    for i in range(3):
        fruit = random.choice(box)
        # Вытащенный фрукт исключаем из корзины
        box.remove(fruit)
        # Если вытащили яблоко
        if fruit == 'apple':
            # Добавляем в список результатов отрицательный результат
            result.append('not_match')
            # Прекращаем эксперимент
            break
        # Если вытащили апельсин
        elif fruit == 'orange':
            # Если апельсинов было меньше 2
            if oranges < 2:         
                # увеличиваем количество вытащенных апельсинов в текущем эксперименте
                oranges += 1
            # Если было 2 апельсина (вытащили 3-й)
            elif oranges == 2:
                # Добавляем в список результатов положительный результат
                result.append('match')
# Выводим долю успешных экспериментов (вытаскивания 3 апельсинов подряд)
result.count('match')/len(result)

0.11976

Вывод - результаты почти совпадают: прогнозное значение - 0.119, полученное - 0.11976

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

### Аналитическое решени задачи

Вероятность вытащить первый раз нестандартную деталь (из 7 стандартных и 3 нестандартных): $P(S_1)=\frac{3}{7+3}=\frac{3}{10}$

Вероятность вытащить на второй раз стандартную деталь (из 7 стандартных и 2 нестандартных): $P(S_3)=\frac{7}{7+2}=\frac{7}{9}$

Вероятность вытащить СНАЧАЛА 1 нестандартную детали, затем стандартную: $P(S_r)=\frac{3}{10}*\frac{7}{9}=\frac{3*7}{10*9}=\frac{7}{30}=0.23333$

In [201]:
# Задаем число экспериментов
n = 100000

# Список результатов экспериментов
result = []

# генерируем выборку
for i in range(n): 
    # На начало эксперимента в ящике 7 стандартных деталей и 3 нестандартных
    box = ['standard']*7 + ['nonstandard']*3
    # Список деталей пуст
    details = []
    # Вытаскиваем детали   
    for i in range(2):
        detail = random.choice(box)
        # Исключаем из ящика вытащенную деталь
        box.remove(detail)
        # Добавляем вытащенную деталь в список
        details.append(detail)
        
    # Если первая деталь нестандартна, а вторая стандартна
    if (details[0] == 'nonstandard') and (details[1] == 'standard'):
        # Отмечаем результат эксперимента как успешный
        result.append('match')
    # В ином случае
    else:
        # Отмечаем результат эксперимента как не успешный
        result.append('not_match')

# Выводим долю успешных экспериментов (сначала вытащили нестандартную деталь, затем стандартную)
result.count('match')/len(result)        

0.23523

Вывод - результаты почти совпадают: прогнозное значение - 0.2333, полученное - 0.23523